In [ ]:
# -*- coding: utf-8 -*-
"""
notebook_workflows_simple.py

Version "notebook-like" (linéaire, très simple) :
1) HW 1F : load -> curve -> pricer -> calibrate -> compare plots -> PFE plot
2) HW 2F : load -> curve -> pricer -> calibrate -> compare plots -> PFE plot
"""

from pathlib import Path
import sys
import numpy as np


# --- imports projet ---
from ir.market.loaders_excel import load_curve_xlsx, load_swaption_template_xlsx
from ir.market.plots import plot_curve, plot_prices_by_tenor, plot_vols_by_tenor
from ir.calibration.vol import black_normal_vol

from ir.pricers.hw1f_pricer import HullWhitePricer
from ir.calibration.hw1f_calibration import HullWhiteCalibrator

from ir.pricers.hw2f_pricer import HullWhite2FPricer
from ir.calibration.hw2f_profile import HullWhite2FProfileCalibrator

from ir.risk.hw2f_sim import HW2FCurveSim
from ir.risk.pfe_swap import pfe_profile_swap
from ir.risk.pfe_plot import plot_pfe_profile


# ============================================================
# CONFIG (modifie juste ça)
# ============================================================
CURVE_XLSX = r"Calibration_Templates\SWPN_Calibration_Template_30092025_USD.xlsx"
TEMPLATE_XLSX = CURVE_XLSX

N_PATHS = 20000
SEED = 2025

PFE_TAU = [0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]
PFE_NOTIONAL = 1_000_000.0
PFE_PAYER = True
PFE_Q = 0.95
PFE_GRID_N = 21


# ============================================================
# 2-3 mini helpers (sinon on répète trop)
# ============================================================
def par_rate(curve, Tau):
    Tau = [float(x) for x in Tau]
    T0, Tn = Tau[0], Tau[-1]
    A0 = 0.0
    for i in range(1, len(Tau)):
        Ti = Tau[i]
        d = Tau[i] - Tau[i - 1]
        A0 += d * float(curve.discount(Ti))
    S0 = (float(curve.discount(T0)) - float(curve.discount(Tn))) / (A0 + 1e-18)
    return A0, S0  # annuity, par swap rate


def ensure_expiry_tenor(df, dates_col="Payment_Dates"):
    if "Expiry" not in df.columns:
        df["Expiry"] = df[dates_col].apply(lambda L: float(L[0]))
    if "Tenor" not in df.columns:
        df["Tenor"] = df[dates_col].apply(lambda L: float(L[-1]) - float(L[0]))


def add_implied_normal_vols_forward_premium(df, curve,
                                           price_col="Price",
                                           model_col="Model_Price",
                                           strike_col="Strike",
                                           dates_col="Payment_Dates"):
    # Convention: forward premium = PV / DF(T0)
    mkt_vol, mdl_vol = [], []
    for _, row in df.iterrows():
        Tau = row[dates_col]
        T0 = float(Tau[0])
        DF0 = float(curve.discount(T0))
        A0, S0 = par_rate(curve, Tau)
        annuity_fwd = A0 / (DF0 + 1e-18)

        strike_pct = float(row[strike_col])      # %
        forward_pct = 100.0 * float(S0)          # %
        notional = float(row.get("Notional", 1.0))

        p_mkt = float(row[price_col])
        p_mdl = float(row[model_col])

        mkt_vol.append(black_normal_vol(p_mkt, forward_pct, strike_pct, T0, notional, annuity_fwd))
        mdl_vol.append(black_normal_vol(p_mdl, forward_pct, strike_pct, T0, notional, annuity_fwd))

    df["Market_Vol (Bps)"] = mkt_vol
    df["Model_Vol (Bps)"] = mdl_vol



# ============================================================
# 1) LOAD DATA
# ============================================================
print("\n=== LOAD curve + swaptions template ===")
curve = load_curve_xlsx(CURVE_XLSX)
swpn = load_swaption_template_xlsx(TEMPLATE_XLSX)

plot_curve(curve, title_prefix="Market")



In [ ]:

# ============================================================
# 2) HW 1F : pricer -> calibrate -> compare -> PFE
# ============================================================
print("\n" + "=" * 80)
print("HW 1F")
print("=" * 80)

print("[1F] init pricer")
pricer_1f = HullWhitePricer(curve, n_paths=N_PATHS, seed=SEED)

print("[1F] calibrate (a,sigma) on swaptions (forward premium)")
mkt_dict = swpn.to_market_dict()
cal_1f = HullWhiteCalibrator(pricer_1f, mkt_dict, calibrate_to="Swaptions")
cal_1f.calibrate(init_a=0.01, init_sigma=0.01)


In [ ]:

print("[1F] compare market vs model (plots)")
df_1f = swpn.with_model_prices_1f(pricer_1f, forward_premium=True)
ensure_expiry_tenor(df_1f)

plot_prices_by_tenor(df_1f, mkt_col="Price", model_col="Model_Price", ylabel="Forward Premium")


In [ ]:

add_implied_normal_vols_forward_premium(df_1f, curve)
plot_vols_by_tenor(df_1f,
                   mkt_col="Market_Vol (Bps)",
                   model_col="Model_Vol (Bps)",
                   ylabel="Normal Vol (Bps)",
                   title="HW1F | Swaption Normal Vols (implied, forward premium)")


In [20]:

print("[1F] PFE (swap)")
A0, K_par = par_rate(curve, PFE_TAU)
grid = np.linspace(0.0, float(PFE_TAU[-1]), int(PFE_GRID_N))

pfe_1f, epe_1f = pfe_profile_swap(
    curve_sim=pricer_1f.curve_sim,
    grid=grid,
    Tau=PFE_TAU,
    K=0.03,
    N=PFE_NOTIONAL,
    payer=PFE_PAYER,
    q=PFE_Q,
)


[1F] PFE (swap)


In [ ]:

# subtitle_1f = f"Tau={PFE_TAU} | N={PFE_NOTIONAL:,.0f} | K(par)={K_par*100:.3f}% | params={pricer_1f.model.parameters}"
plot_pfe_profile(grid, pfe_1f, epe=epe_1f, q=PFE_Q, title="HW1F | PFE profile (swap)")



In [ ]:

# ============================================================
# 3) HW 2F : pricer -> calibrate -> compare -> PFE
# ============================================================
print("\n" + "=" * 80)
print("HW 2F (G2++)")
print("=" * 80)

print("[2F] init pricer")
pricer_2f = HullWhite2FPricer(curve)

print("[2F] calibrate profile on swaptions (forward premium)")
cal_2f = HullWhite2FProfileCalibrator(pricer_2f, mkt_dict, use_forward_premium=True)

# grids "petites" (lisible). Tu élargiras ensuite.
grid_a = [0.01, 0.02, 0.05, 0.10, 0.20]
grid_b = [0.001, 0.003, 0.01, 0.02, 0.05]
grid_rho = [-0.8, -0.5, -0.2, 0.0, 0.2]

cal_2f.calibrate_profile(
    grid_a=grid_a,
    grid_b=grid_b,
    grid_rho=grid_rho,
    init_sigma=0.01,
    init_eta=0.008,
    verbose_inner=False,
    top_k=3,
)


In [ ]:

print("[2F] compare market vs model (plots)")
df_2f = swpn.with_model_prices_2f(pricer_2f, forward_premium=True)
ensure_expiry_tenor(df_2f)

plot_prices_by_tenor(df_2f, mkt_col="Price", model_col="Model_Price", ylabel="Forward Premium")


In [ ]:

add_implied_normal_vols_forward_premium(df_2f, curve)
plot_vols_by_tenor(df_2f,
                   mkt_col="Market_Vol (Bps)",
                   model_col="Model_Vol (Bps)",
                   ylabel="Normal Vol (Bps)",
                   title="HW2F | Swaption Normal Vols (implied, forward premium)")


In [25]:

print("[2F] PFE (swap)")
curve_sim_2f = HW2FCurveSim(
    curve=curve,
    model=pricer_2f.model,
    n_paths=N_PATHS,
    seed=SEED,
    use_legacy_global_seed=True,
)

pfe_2f, epe_2f = pfe_profile_swap(
    curve_sim=curve_sim_2f,
    grid=grid,
    Tau=PFE_TAU,
    K=0.03,
    N=PFE_NOTIONAL,
    payer=PFE_PAYER,
    q=PFE_Q,
)


[2F] PFE (swap)


In [ ]:

# subtitle_2f = f"Tau={PFE_TAU} | N={PFE_NOTIONAL:,.0f} | K(par)={K_par*100:.3f}% | params={pricer_2f.model.parameters}"
plot_pfe_profile(grid, pfe_2f, epe=epe_2f, q=PFE_Q, title="HW2F | PFE profile (swap)")
